In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import os
import argparse
from torch.utils.data.sampler import SubsetRandomSampler
from models import *
from utils import progress_bar
from tqdm import tqdm
import wandb

In [2]:
wandb.init(
    project="cifar-10",
    config={"lr":0.01,
            "epoch":5,
            "architecture": "VGG13",
            "dataset": "CIFAR-10",}
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: huang479016264. Use `wandb login --relogin` to force relogin


In [4]:
parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
#change the below default value  to tune hyper parameters
parser.add_argument('--lr', default= .01, type=float, help='learning rate')
parser.add_argument('--resume', '-r', action='store_true', help='resume from checkpoint')
args = parser.parse_args(args=[])


device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_val = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
val_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_val)


num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(0.2 * num_train))
train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)



trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, num_workers=2, sampler= train_sampler)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=128,  num_workers=2, sampler=valid_sampler)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
net = VGG('VGG13')
#net = LeNet()
#net = ResNet18()
#net = PreActResNet18()
#net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
#net = SENet18()
# net = ShuffleNetV2(1)
net = EfficientNetB0()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if args.resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt_tqdm.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)



# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        wandb.log({"Acc_train":100.*correct/total})
        wandb.log({"Loss_train:":train_loss/(batch_idx+1)})
#testing
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            #print('entering')
            progress_bar(batch_idx, len(testloader))
            

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt_tqdm.pth')
        best_acc = acc
        print('testing Accuracy: ', best_acc)
        wandb.log({"Acc_test":best_acc})

#validation
def validation(epoch):
    best_acc_val = 0
    net.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(valloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            #print('entering')
            progress_bar(batch_idx, len(valloader))
            
    # Save checkpoint.
    acc_val = 100.*correct/total
    if acc_val > best_acc_val :
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc_val,
            'epoch': epoch,
        }
        best_acc_val = acc_val
        print('validation Accuracy: ', best_acc_val)
        wandb.log({"validation Accuracy":best_acc_val})

for epoch in tqdm(range(start_epoch, start_epoch+5)):
    train(epoch)
    validation(epoch)
    test(epoch)

wandb.finish()

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
==> Building model..


  0%|          | 0/5 [00:00<?, ?it/s]


Epoch: 0
 [================================================================>]  Step: 1s238ms | Tot: 22s795ms | Loss: 1.794 | Acc: 36.065% (14426/4000 313/313 .....................................................]  Step: 68ms | Tot: 203ms | Loss: 2.545 | Acc: 14.323% (55/38 3/313 >................................................................]  Step: 69ms | Tot: 339ms | Loss: 2.554 | Acc: 14.219% (91/64 5/313 =>...............................................................]  Step: 61ms | Tot: 549ms | Loss: 2.570 | Acc: 13.281% (136/102 8/313 =>...............................................................]  Step: 68ms | Tot: 617ms | Loss: 2.546 | Acc: 13.542% (156/115 9/313 =>...............................................................]  Step: 64ms | Tot: 681ms | Loss: 2.548 | Acc: 13.438% (172/128 10/313 ==>..............................................................]  Step: 62ms | Tot: 744ms | Loss: 2.532 | Acc: 13.849% (195/140 11/313 ==>....................................

 20%|██        | 1/5 [00:48<03:14, 48.61s/it]

Saving..
testing Accuracy:  45.78

Epoch: 1
 [================================================================>]  Step: 45ms | Tot: 21s263ms | Loss: 1.463 | Acc: 48.405% (19362/4000 313/313 ........................................................]  Step: 62ms | Tot: 285ms | Loss: 1.505 | Acc: 42.500% (272/64 5/313 =>...............................................................]  Step: 67ms | Tot: 353ms | Loss: 1.511 | Acc: 42.969% (330/76 6/313 =>...............................................................]  Step: 66ms | Tot: 420ms | Loss: 1.484 | Acc: 44.085% (395/89 7/313 =>...............................................................]  Step: 65ms | Tot: 551ms | Loss: 1.529 | Acc: 43.229% (498/115 9/313 =>...............................................................]  Step: 67ms | Tot: 618ms | Loss: 1.546 | Acc: 43.359% (555/128 10/313 ==>..............................................................]  Step: 63ms | Tot: 682ms | Loss: 1.537 | Acc: 43.963% (619/140 11/313 ==>.

...

 80%|████████  | 4/5 [03:07<00:46, 46.60s/it]

Saving..
testing Accuracy:  63.12

Epoch: 4
 [================================================================>]  Step: 49ms | Tot: 21s369ms | Loss: 1.028 | Acc: 63.735% (25494/4000 313/313  .......................................................]  Step: 61ms | Tot: 281ms | Loss: 1.097 | Acc: 60.312% (386/64 5/313 =>...............................................................]  Step: 65ms | Tot: 347ms | Loss: 1.116 | Acc: 59.896% (460/76 6/313 =>...............................................................]  Step: 60ms | Tot: 479ms | Loss: 1.083 | Acc: 60.742% (622/102 8/313 =>...............................................................]  Step: 66ms | Tot: 545ms | Loss: 1.088 | Acc: 60.417% (696/115 9/313 =>...............................................................]  Step: 66ms | Tot: 612ms | Loss: 1.091 | Acc: 60.156% (770/128 10/313 ==>..............................................................]  Step: 67ms | Tot: 679ms | Loss: 1.092 | Acc: 60.227% (848/140 11/313 ==>

100%|██████████| 5/5 [03:53<00:00, 46.69s/it]

Saving..
testing Accuracy:  66.07


Acc_test,▁▄▆▇█
Acc_train,▁▂▂▃▃▄▄▄▅▅▅▆▆▆▆▆▇▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████
Loss_train:,█▇▆▆▅▅▅▅▃▃▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
validation Accuracy,▁▃▅▇█
Acc_test,66.07
Acc_train,63.735
Loss_train:,1.02788
validation Accuracy,64.31
